Real Time Radar Parameter Optimizer
===================================

Carson Anderson & Calvin Henggeler  
ASEN 5264 Decision Making Under Uncertainty - Spring 2025  
Semester Project  

In [ ]:
using POMDPs
using POMDPTools: transition_matrices, reward_vectors, SparseCat, Deterministic, RolloutSimulator, DiscreteBelief, FunctionPolicy, ordered_states, ordered_actions, DiscreteUpdater, transition, UnderlyingMDP, implicitDistribution
using QuickPOMDPs: QuickPOMDP
using POMDPModels: TigerPOMDP, TIGER_LEFT, TIGER_RIGHT, TIGER_LISTEN, TIGER_OPEN_LEFT, TIGER_OPEN_RIGHT
using NativeSARSOP: SARSOPSolver
using POMDPTesting: has_consistent_distributions
using QMDP: QMDPSolver
using Plots
using Statistics: std
using POMDPPolicies: alphavectors, FunctionPolicy
using Random
using radarSimulator 
include("radarSimulator.jl")

## 1. Create Smart Radar POMDP

In [ ]:
radar_tracking = QuickPOMDP(
    
    # --- Environment Parameters ---
    c = 3*10^8  # Speed of light
    f = 1e9     # Frequency
    λ = c/f     # Wavelength
    x_coverage_area = 100e3     # 100 km x 100 km area
    y_coverage_area = 100e3
    radar_location = (50e3, 0)  # Radar location (x, y) in meters

    # --- STATE SPACE ---
    states = nothing,     # We'll model state generation implicitly
    
    # --- ACTION SPACE ---
    actions = [
        # Example discretized radar settings (fill in realistic values)
        # start with a range of differeent realistic settings
        (beamwidth=5, pulse_length=0.000001, Tipp= 0.001 ,rotation_rate=5),
        (beamwidth=5, pulse_length=0.000001, Tipp= 0.001 ,rotation_rate=10),
        (beamwidth=5, pulse_length=0.000002, Tipp= 0.001 ,rotation_rate=5),
        (beamwidth=5, pulse_length=0.000002, Tipp= 0.001 ,rotation_rate=10),
    ],
    
    obstype = Vector{Float64},   # Observation: [power, phase, delay, doppler]

    gen = function (s, a, rng)
        # s = (x, y, vx, vy),   extract sate information
        x, y, vx, vy = s

        true_range = sqrt((xp - radar_location[1])^2 + (yp - radar_location[2])^2)

        # Observation Measurments
        received_power  = radar_return_power(λ, true_range)
        view_angle      = radar_view_angle(x, y, radar_location)
        delay           = radar_return_delay(true_range)
        doppler_obs     = radar_doppler(s)
        measurments = [received_power, view_angle, delay, doppler_obs]

        # solve for the observed state
        o = radar_meas_2_obs_sate()

        # REWARD: - (tracking error) - (action cost)
        xo, yo, vxo, vyo = o
        # tracking_error = sqrt(xp^2 + yp^2)
        # action_penalty = 0.1*(a.beamwidth + a.pulse_length + abs(a.steering_angle)/30)
        r = -tracking_error - action_penalty

        # TRANSITION MODEL: Target motion (simple linear)
        dt = 0.1            # time step
        xp = x + vx*dt      # + noise_std*randn(rng)
        yp = y + vy*dt      # + noise_std*randn(rng)
                            # keep velocities
        sp = (xp, yp, vx, vy)

        return (sp=sp, o=o, r=r)
    end,

    # change this later to start at edge of the radar coverage area
    initialstate = ImplicitDistribution(rng -> (
        x = 0,
        y = 50e3,
        vx = 200.0*randn(rng),
        vy = 10.0*randn(rng)
    )),

    discount = 0.99,

    isterminal = s -> false  # Tracking problem — no natural terminal state 
    # TODO: termainal states will be when the target reaches the end of the radar simulator
)


## 2. Create Updater (Particle Filter)